In [1]:
import urllib
import zlib
import os
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
%config Completer.use_jedi = False

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
import gzip

One year of monthly weather data was downloaded in the folder "ObservedData" for 2003 
from Meteo France Archives of in situ observations.

### Finding Your Weather staion

In [4]:
## list of french weather stations file is in github
link=r'D:\Weather_files\Weather data for South of France\EssentialWeatherStationsList.csv'

In [5]:
FrenchWeatherStations = pd.read_csv(link,sep=';')
FrenchWeatherStations.tail(4)

,ID,Nom,Latitude,Longitude,Altitude
58,81405,CAYENNE-MATOURY,4.822333,-52.365333,4
59,81408,SAINT GEORGES,3.890667,-51.804667,6
60,81415,MARIPASOULA,3.640167,-54.028333,106
61,89642,DUMONT D'URVILLE,-66.663167,140.001000,43


In [6]:
FrenchWeatherStations.head(60)

,ID,Nom,Latitude,Longitude,Altitude
0,7005,ABBEVILLE,50.136000,1.834000,69
1,7015,LILLE-LESQUIN,50.570000,3.097500,47
2,7020,PTE DE LA HAGUE,49.725167,-1.939833,6
3,7027,CAEN-CARPIQUET,49.180000,-0.456167,67
4,7037,ROUEN-BOOS,49.383000,1.181667,151
5,7072,REIMS-PRUNAY,49.209667,4.155333,95
6,7110,BREST-GUIPAVAS,48.444167,-4.412000,94
7,7117,PLOUMANAC'H,48.825833,-3.473167,55
8,7130,RENNES-ST JACQUES,48.068833,-1.734000,36
9,7139,ALENCON,48.445500,0.110167,143


In [7]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [8]:
#Coordinates of target location
location_latitude=47.236596
location_longitude=-1.559562

distances=[]

for i in range(len(FrenchWeatherStations)):
    lats = FrenchWeatherStations['Latitude'][i]
    lons = FrenchWeatherStations['Longitude'][i]
    distance = calculate_distance(location_latitude,location_longitude,lats,lons)
    distances.append(distance)
    
min_value=min(distances)
index_min=distances.index(min_value)
ID_min=FrenchWeatherStations['ID'][index_min]
Name=FrenchWeatherStations['Nom'][index_min]

print('Name of closest weather station is {}'.format(Name))
print('ID of closest weather station is {} and its distance is {:.2f} km'.format(ID_min,min_value))

Name of closest weather station is NANTES-BOUGUENAIS
ID of closest weather station is 7222 and its distance is 10.33 km


### Decompressing and filtering for Nantes=7222 .gz downloaded grided weatherfiles

In [9]:
## Observed weather data from meteoFrance archives
## 
cwd = os.path.abspath(r'D:\Weather_files\ObservedData2003') 
files = os.listdir(cwd)

In [10]:
df = pd.DataFrame()
for file in files:
    if file.endswith('.gz'):
        f=gzip.open('D:/Weather_files/ObservedData2003/'+str(file), 'rb')
        df_W=pd.read_csv(f,delimiter=';')
        df_output = df_W[df_W.numer_sta == ID_min] # weather station ID found earlier
        df_output.loc[:,"date"] = pd.to_datetime(df_output["date"], format = "%Y%m%d%H%M%S")
        df_output.index = df_output["date"]
        df_output=df_output.drop(["date", "numer_sta"], axis = 1)
        df_output.to_csv('D:/Weather_files/ObservedData2003/'+str(file)+".csv")
        df = df.append(df_output)

C:\Users\obaidullah.yaqubi\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### Transforming three hour weather data to hourly and creating an epw weather file from the given data

In [11]:
#df.head()

In [12]:
#'date'       :Datetime 
#'numer_sta'  :number of weather station (Nantes = 7222)
#'pmer'     :PRESSURE AT SEA LEVEL (Pression au niveau mer)[pa]
#'tend '    :presssure variations in every 3 hours(Variation de pression en 3 heures)[pa]
#'cod_tend ':Type de tendance barométrique [code (0200)]
#'dd'       :WIND DIRECTION (Direction du vent moyen 10 mn [degrees])
#'ff'       :WIND SPEED (Vitesse du vent moyen 10 mn [m/s])
#'t'        :DRY BULB TEMPERATURE (Température [K])
#'td'       :DEW POINT TEMPERATURE (Point de rosée [K])
#'u',        :REATIVE HUMIDITY (Humidité [%])
#'vv',       :HORIZONTAL VISBILITY (Visibilité horizontale [m])
#'ww',       :Temps présent [code (4677)]
#'w1',       :Temps passé 1 [code (4561)]
#'w2',       :Temps passé 2 [code (4561)]
#'n',        :TOTAL CLOUD COVER (Nebulosité totale [%])
#'nbas',     :OPAQUE SKY COVER measure (1/10) (Nébulosité des nuages de l'étage inférieur [octa] measured (1/8))
#'hbas',     :Hauteur de la base des nuages de l'étage inférieur [m]
#'cl',       :Type des nuages de l'étage inférieur
#'cm',       :Type des nuages de l'étage moyen
#'ch',       :Type des nuages de l'étage supérieur
#'pres',     :ATMOSPHERIC STATION PRESSURE (Pression station [pa])
#'niv_bar',  :Niveau barométrique [pa]
#'geop',     :Géopotentiel [m2/s2]
#'tend24',   :Variation de pression en 24 heures [pa]
#'tn12',     :Température minimale sur 12 heures [K]
#'tn24',     :Température minimale sur 24 heures [K]
#'tx12',     :Température maximale sur 12 heures [K]
#'tx24',     :Température maximale sur 24 heures [K]
#'tminsol',  :Température minimale du sol sur 12 heures [K]
#'sw',       :Méthode mesure tw 
#'tw',       :Wet thermometer temperature (Température du thermomètre mouillé [K])
#'raf10',    :Rafales sur les 10 dernières minutes [m/s]
#'rafper',   :Rafales sur une période [m/s]
#'per',      :Période de mesure de la rafale [min]
#'etat_sol', :Etat du sol
#'ht_neige', :Hauteur totale de la couche de neige, glace, autre au sol [m]
#'ssfrai',   :Hauteur de la neige fraîche [m]
#'perssfrai',:Periode de mesure de la neige fraiche [1/10 hour]
#'rr1',      :LIQUID PRECIPITATION DEPTH (Précipitations dans les 1 dernières heures [mm])
#'rr3',      : =
#'rr6',      : =
#'rr12',     : =
#'rr24',     : =
#'phenspe1', :Phénomène spécial 1 [code (3778)]
#'phenspe2', : =
#'phenspe3', : =
#'phenspe4', : =
#'nnuage1',  :Nébulosité cche nuageuse N [octa] 
#'ctype1',   :Type nuage N [code (0500)]
#'hnuage1',  :Hauteur de base N [m]
#'nnuage2',  : =
#'ctype2',   : =
#'hnuage2',  : =
#'nnuage3',  : =
#'ctype3',   : =
#'hnuage3',  : =
#'nnuage4',  : =
#'ctype4',   : =
#'hnuage4',  : =
#'Unnamed: 59':

In [13]:
## replace mq to NaN 
df = df.replace('mq', np.nan, regex=True)

In [14]:
df=df.resample('H').last()# resampling doesn't go past the last index. 
                          #Parameters of weather file from MeteoFrance archives

In [15]:
df['t'] = df.t.astype(float)
df['td'] = df.td.astype(float)
df['u'] = df.u.astype(float)
df['pres'] = df.pres.astype(float)
df['n'] = df.n.astype(float)
df['dd'] = df.dd.astype(float)
df['ff'] = df.ff.astype(float)

In [16]:
##Interpolation of 3hr data to hourly weather data 
#dry bulb temperature 
df['t'].interpolate(method='polynomial', order=7,limit_direction='both',limit=2, inplace=True)
# dew point temperature
df['td'].interpolate(method='polynomial', order=7,limit_direction='both',limit=2, inplace=True)
# relative humidity 
df['u'].interpolate(method='linear', limit_direction='both',limit=2, inplace=True)
# Atmospheric pressure 
df['pres'].interpolate(method='linear', limit_direction='both',limit=2, inplace=True)
# Cloud cover /total /opaque SKY cover 
df['n'].interpolate(method='linear', limit_direction='both',limit=2, inplace=True)
## Wind direction 
df['dd'].interpolate(method='linear', limit_direction='both',limit=2, inplace=True)
## Wind speed 
df['ff'].interpolate(method='linear', limit_direction='both',limit=2, inplace=True)

In [17]:
## year, month, day, hour to dataframe from index 
df['Year']=df.index.year
df['Month']=df.index.month
df['Day']=df.index.day
df['Hour']=df.index.hour

In [18]:
# TOtal cloud cover is measured in 1/10
df['TotalCloudCover']=round(df['n']//10,0)
# Opaque cloud cover is measured in 1/9
df['OpaqueCloudCover']=round(df['n']//11.2,0)

In [19]:
#df.head()

In [20]:
df.columns

Index(['pmer', 'tend', 'cod_tend', 'dd', 'ff', 't', 'td', 'u', 'vv', 'ww',
       'w1', 'w2', 'n', 'nbas', 'hbas', 'cl', 'cm', 'ch', 'pres', 'niv_bar',
       'geop', 'tend24', 'tn12', 'tn24', 'tx12', 'tx24', 'tminsol', 'sw', 'tw',
       'raf10', 'rafper', 'per', 'etat_sol', 'ht_neige', 'ssfrai', 'perssfrai',
       'rr1', 'rr3', 'rr6', 'rr12', 'rr24', 'phenspe1', 'phenspe2', 'phenspe3',
       'phenspe4', 'nnuage1', 'ctype1', 'hnuage1', 'nnuage2', 'ctype2',
       'hnuage2', 'nnuage3', 'ctype3', 'hnuage3', 'nnuage4', 'ctype4',
       'hnuage4', 'Unnamed: 59', 'Year', 'Month', 'Day', 'Hour',
       'TotalCloudCover', 'OpaqueCloudCover'],
      dtype='object')

In [21]:
##Rearranging columns and selecting only a few that we use in modification of weather file
Columns=['Year', 'Month', 'Day', 'Hour','dd', 'ff', 't', 'td','u','n', 
         'pres','TotalCloudCover', 'OpaqueCloudCover']
df=df[Columns]
#  year is not a leap year, if you dealing with a leap year, in order to make it fit with typical
#  weather file, the leap day needs to be removed. (typical weather years are not leap years)
# Adding two more hours to the dataframe 
df.reset_index(inplace=True)

In [22]:
df2=pd.DataFrame({'Day':[31.0,31.5],
                 'Hour':[22.0,23.0],
                 'Month':[12.0,12.0]})
df=df.append(df2)

C:\Users\obaidullah.yaqubi\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:7134: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return concat(


In [23]:
df.reset_index(drop=True,inplace=True)
df.tail()

,Day,Hour,Month,OpaqueCloudCover,TotalCloudCover,Year,date,dd,ff,n,pres,t,td,u
8755,31.0,19.0,12.0,5.0,6.0,2003.0,2003-12-31 19:00:00,NaN,NaN,60.0,102353.333333,275.663445,273.814165,92.333333
8756,31.0,20.0,12.0,5.0,6.0,2003.0,2003-12-31 20:00:00,NaN,NaN,60.0,102316.666667,276.485598,273.771780,93.666667
8757,31.0,21.0,12.0,NaN,NaN,2003.0,2003-12-31 21:00:00,NaN,NaN,NaN,102280.000000,273.050000,272.450000,95.000000
8758,31.0,22.0,12.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8759,31.5,23.0,12.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Writing data from dataframe to EPW weather file for 2003

In [24]:
df2=df

In [25]:
## missing data are filled with automatic closest neighbor 
df2=df2.fillna(method="ffill")

In [26]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
Day                 8760 non-null float64
Hour                8760 non-null float64
Month               8760 non-null float64
OpaqueCloudCover    8760 non-null float64
TotalCloudCover     8760 non-null float64
Year                8760 non-null float64
date                8760 non-null datetime64[ns]
dd                  8760 non-null float64
ff                  8760 non-null float64
n                   8760 non-null float64
pres                8760 non-null float64
t                   8760 non-null float64
td                  8760 non-null float64
u                   8760 non-null float64
dtypes: datetime64[ns](1), float64(13)
memory usage: 958.2 KB


In [27]:
df2.head(2)

,Day,Hour,Month,OpaqueCloudCover,TotalCloudCover,Year,date,dd,ff,n,pres,t,td,u
0,1.0,0.0,1.0,8.0,9.0,2003.0,2003-01-01 00:00:00,180.000000,6.200000,90.0,100940.0,283.750000,282.850000,94.000000
1,1.0,1.0,1.0,8.0,9.0,2003.0,2003-01-01 01:00:00,183.333333,7.033333,90.0,100870.0,285.003135,284.253876,94.333333


In [28]:
#df2['TotalCloudCover'] = df2.TotalCloudCover.astype(int)
#df2['OpaqueCloudCover']=df2.OpaqueCloudCover.astype(int)
df2[['Day','Hour','Month','Year']] = df2[['Day','Hour','Month','Year']].astype(int)
df2['WindDirection']=df2['dd'].astype(int)
df2['DryBulbTemp']=df2['t']-273.15
df2['DewPointTemp']=df2['td']-273.15

In [29]:
## Writing the weather data on epw weather template
from ladybug.epw import EPW
from pandas import DataFrame
from collections import OrderedDict
%config Completer.use_jedi = False

In [30]:
## Checking epw weather file data 
epwFile=EPW(r'D:\Weather_files\ObservedData2003\Nantes_2000_2019_Contemporary.epw')
epwDataList=epwFile.to_dict()['data_collections']
epwDataDict = OrderedDict()

for dataColumns in epwDataList:
    dataName=dataColumns['header']['data_type']['name']
    epwDataDict[dataName]=dataColumns['values']

epwDataFrame = DataFrame(epwDataDict)
epwDataFrame.head(3)

,Year,Month,Day,Hour,Minute,Uncertainty Flags,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,Extraterrestrial Horizontal Radiation,Extraterrestrial Direct Normal Radiation,Horizontal Infrared Radiation Intensity,Global Horizontal Radiation,Direct Normal Radiation,Diffuse Horizontal Radiation,Global Horizontal Illuminance,Direct Normal Illuminance,Diffuse Horizontal Illuminance,Zenith Luminance,Wind Direction,Wind Speed,Total Sky Cover,Opaque Sky Cover,Visibility,Ceiling Height,Present Weather Observation,Present Weather Codes,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
0,2005,12,31,24,60,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,5.7,5.5,98,100879,0,0,312,0,0,0,0,0,0,9999,223,1.3,10,9,9999.0,99999,9,999999999,14,0.103,0,5,0.2,1.7,99.0
1,2005,1,1,1,60,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,11.0,9.2,88,100879,0,0,332,0,0,0,0,0,0,9999,218,2.4,10,9,9999.0,99999,9,999999999,18,0.092,0,4,0.2,1.5,99.0
2,2005,1,1,2,60,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,11.1,8.0,81,100879,0,0,327,0,0,0,0,0,0,9999,252,1.9,10,9,9999.0,99999,9,999999999,16,0.092,0,4,0.2,0.0,99.0


In [31]:
# Name and indices of columns in a typical epw weather file
#field_number: index value between 0 to 34 for different available epw fields.
#            0 Year
#            1 Month
#            2 Day
#            3 Hour
#            4 Minute

#            6 Dry Bulb Temperature
#            7 Dew Point Temperature
#            8 Relative Humidity
#            9 Atmospheric Station Pressure
#            10 Extraterrestrial Horizontal Radiation
#            11 Extraterrestrial Direct Normal Radiation
#            12 Horizontal Infrared Radiation Intensity
#            13 Global Horizontal Radiation
#            14 Direct Normal Radiation
#            15 Diffuse Horizontal Radiation
#            16 Global Horizontal Illuminance
#            17 Direct Normal Illuminance
#            18 Diffuse Horizontal Illuminance
#            19 Zenith Luminance
#            20 Wind Direction
#            21 Wind Speed
#            22 Total Sky Cover
#            23 Opaque Sky Cover
#            24 Visibility
#            25 Ceiling Height
#            26 Present Weather Observation
#            27 Present Weather Codes
#            28 Precipitable Water
#            29 Aerosol Optical Depth
#            30 Snow Depth
#            31 Days Since Last Snowfall
#            32 Albedo
#            33 Liquid Precipitation Depth
#            34 Liquid Precipitation Quantity

In [32]:
def changeEPWData(oldEpwFilePath,newEpwFilePath,dataIndex,dataList):
    with open(oldEpwFilePath) as oldStream,open(newEpwFilePath,"w") as newStream:
        numCount=0
        for idx,lines in enumerate(oldStream):
            if lines.strip():
                try:
                    lineSplit=lines.strip().split(",")
                    dataTest=float(lineSplit[0])
                    lineSplit[dataIndex]=str(dataList[numCount])
                    data=",".join(lineSplit)
                    newStream.write(data+"\n")
                    numCount+=1
                except ValueError:
                    newStream.write(lines.strip()+"\n")
            else:
                newStream.write(lines)
    return newEpwFilePath

In [33]:
#Writing column by column of weather file to make sure the structure of EPW file doesn't change
#a user can decide to modify one climate variable or multiples climate variables

In [34]:
# In this script a temporary folder was created to save modified versions of EPW weather file.
# With every new climate variable, previously modified file from temporary folder is selected and
# further overwritten until all climate variables are overwritten. 

In [35]:
### Replacing year
changeEPWData(r'D:\Weather_files\ObservedData2003\Nantes_2000_2019_Contemporary.epw',
              r'D:\Weather_files\ObservedData2003\Temporary\year.epw',
              dataIndex=0,
              dataList=df2.Year)

#changeEPWData(r'D:\Weather_files\ObservedData\Temporary\year.epw',
#              r'D:\Weather_files\ObservedData\Temporary\month.epw',
#              dataIndex=1,
#              dataList=df2.Month)
#changeEPWData(r'D:\Weather_files\ObservedData\Temporary\month.epw',
#              r'D:\Weather_files\ObservedData\Temporary\day.epw',
#              dataIndex=2,
#              dataList=df2.Day)
#changeEPWData(r'D:\Weather_files\ObservedData\Temporary\day.epw',
#              r'D:\Weather_files\ObservedData\Temporary\hour.epw',
#              dataIndex=3,
#              dataList=df2.Hour)
# Substituting DBT
changeEPWData(r'D:\Weather_files\ObservedData2003\Temporary\year.epw',
              r'D:\Weather_files\ObservedData2003\Temporary\DryBTemp.epw',
              dataIndex=6,
              dataList=df2.DryBulbTemp)
# Substituting DewPoint
changeEPWData(r'D:\Weather_files\ObservedData2003\Temporary\DryBTemp.epw',
              r'D:\Weather_files\ObservedData2003\Temporary\DewPTemp.epw',
              dataIndex=7,
              dataList=df2.DewPointTemp)
#Substitutinh RH
changeEPWData(r'D:\Weather_files\ObservedData2003\Temporary\DewPTemp.epw',
              r'D:\Weather_files\ObservedData2003\Temporary\RH.epw',
              dataIndex=8,
              dataList=df2.u)
#Substituting Atm pressure
changeEPWData(r'D:\Weather_files\ObservedData2003\Temporary\RH.epw',
              r'D:\Weather_files\ObservedData2003\Temporary\AtmPressure.epw',
              dataIndex=9,
              dataList=df2.pres)

## Replacing wind direction
changeEPWData(r'D:\Weather_files\ObservedData2003\Temporary\AtmPressure.epw',
              r'D:\Weather_files\ObservedData2003\Temporary\WindDirection.epw',
              dataIndex=20,
              dataList=df2.WindDirection)

## Replacing wind speed
changeEPWData(r'D:\Weather_files\ObservedData2003\Temporary\WindDirection.epw',
              r'D:\Weather_files\ObservedData2003\Observed_Nantes_2003.epw',
              dataIndex=21,
              dataList=df2.ff)

'D:\\Weather_files\\ObservedData2003\\Observed_Nantes_2003.epw'

#### Verifying how it looks

In [36]:
# Checking how it looks now
epwFile=EPW(r'D:\Weather_files\ObservedData2003\Observed_Nantes_2003.epw')
epwDataList=epwFile.to_dict()['data_collections']
epwDataDict = OrderedDict()

for dataColumns in epwDataList:
    dataName=dataColumns['header']['data_type']['name']
    epwDataDict[dataName]=dataColumns['values']

epwDataFrame1 = DataFrame(epwDataDict)
epwDataFrame1.head()

,Year,Month,Day,Hour,Minute,Uncertainty Flags,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,Extraterrestrial Horizontal Radiation,Extraterrestrial Direct Normal Radiation,Horizontal Infrared Radiation Intensity,Global Horizontal Radiation,Direct Normal Radiation,Diffuse Horizontal Radiation,Global Horizontal Illuminance,Direct Normal Illuminance,Diffuse Horizontal Illuminance,Zenith Luminance,Wind Direction,Wind Speed,Total Sky Cover,Opaque Sky Cover,Visibility,Ceiling Height,Present Weather Observation,Present Weather Codes,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
0,2003,12,31,24,60,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,-0.100000,-0.700000,95,100940,0,0,312,0,0,0,0,0,0,9999,40,2.100000,10,9,9999.0,99999,9,999999999,14,0.103,0,5,0.2,1.7,99.0
1,2003,1,1,1,60,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,10.600000,9.700000,94,100870,0,0,332,0,0,0,0,0,0,9999,180,6.200000,10,9,9999.0,99999,9,999999999,18,0.092,0,4,0.2,1.5,99.0
2,2003,1,1,2,60,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,11.853135,11.103876,94,100800,0,0,327,0,0,0,0,0,0,9999,183,7.033333,10,9,9999.0,99999,9,999999999,16,0.092,0,4,0.2,0.0,99.0
3,2003,1,1,3,60,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,11.626363,10.860776,95,100730,0,0,325,0,0,0,0,0,0,9999,186,7.866667,10,9,9999.0,99999,9,999999999,16,0.092,0,4,0.2,0.0,99.0
4,2003,1,1,4,60,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,11.200000,10.400000,95,100620,0,0,324,0,0,0,0,0,0,9999,190,8.700000,10,9,9999.0,99999,9,999999999,16,0.092,0,4,0.2,0.0,99.0
